In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, Astra kekgun example

This is the class method for running Xopt.

In [ ]:
from xopt import Xopt

# Notebook printing output
from xopt import output_notebook

output_notebook()

In [ ]:
import yaml
from xopt import Xopt

YAML = """
xopt:
  output_path: temp

algorithm:
  name: cnsga
  options:
    max_generations: 3
    population_size: 8
    crossover_probability: 0.9
    mutation_probability: 1.0
    selection: auto
    population: null
    show_progress: True

simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options:
    astra_input_file: templates/kekgun/kekgun.in
    generator_input_file: templates/kekgun/dist004.in

# Note that you can call another file in the top level group:
vocs:
    variables:
      sig_x: [0.05, 1]
      lt: [0.005, 0.07]
      maxe(1): [20, 50]
      phi(1): [-30, 30]
      maxb(1): [0, 0.4]
      maxe(2): [0, 32]
      phi(2): [-180, 180]
      maxb(2): [0, 0.3]
      maxe(3): [0, 32]
      maxe(4): [0, 32]
      phi(3): [-45, 45]
      phi(4): [-45, 45]
      phi(6): [-45, 45]
    constants:
      ipart: 1000
      lspch: true
      zstop: 16.54

    objectives:
      end_core_emit_95percent_x: MINIMIZE
      end_sigma_z: MINIMIZE
    constraints:
      end_sigma_z: [LESS_THAN, 0.0015]
      end_core_emit_95percent_x: [LESS_THAN, 9.0e-07]
      end_sigma_energy: [LESS_THAN, 200000.0]
      end_higher_order_energy_spread: [LESS_THAN, 5000.0]
      end_mean_kinetic_energy: [GREATER_THAN, 90000000.0]
      end_n_particle_loss: [LESS_THAN, 1]
    linked_variables: null

"""

!mkdir temp/

In [ ]:
# Create object
X = Xopt(YAML)

# Change some things to make it run faster
X.vocs["constants"]["lspch"] = True
X.vocs["constants"]["ipart"] = 1000
X.vocs["constants"]["zstop"] = 0.2

# Show config
X

In [ ]:
# Check random inputs
X.random_inputs()

In [ ]:
# Evaluate with some particular settings
X.evaluate({"ipart": 1000, "lspch": True, "zstop": 0.2})

In [ ]:
%%time
# Do a random evaluate to check that everything will run
output = X.random_evaluate()
output

In [ ]:
# These are the algorithm options
X.algorithm["options"]

In [ ]:
# These are the options in the evaluate function
X.simulation["options"]

# Run CNSGA using processes or threads

In [ ]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor

# from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [ ]:
# Create object
X = Xopt(YAML)

# Change some things to make it run faster
X.vocs["constants"]["lspch"] = False
X.vocs["constants"]["ipart"] = 100
X.vocs["constants"]["zstop"] = 0.2
X.results

In [ ]:
# Run
X.run(executor=executor)

In [ ]:
# Check for errors
X.results["error"]

# Write this configuration

In [ ]:
X.config["algorithm"]["options"]["population"] = "temp/pop_3.json"
X.save("test.yaml")

# Run with MPI

In [ ]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.mpi.run -vv --logfile xopt.log test.yaml

# Dask 

In [ ]:
from dask.distributed import Client

executor = Client()
# executor = Client(processes=True)
executor

In [ ]:
# Wait a few seconds for the Dask cluster to start
from time import sleep

sleep(5)

In [ ]:
X.algorithm["options"]["max_generations"] = 4
X.algorithm["options"]["population_size"] = 32
X.algorithm["options"]["population"] = None
X.results = None
X

In [ ]:
# Run again
X.run(executor=executor)
executor.close()

In [ ]:
X.results["error"]

# Plot results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

kx = "end_sigma_z"
ky = "end_core_emit_95percent_x"

x = np.array([d[kx] for d in X.results["outputs"]])
y = np.array([d[ky] for d in X.results["outputs"]])


plt.scatter(x, y)

In [ ]:
# X.results['outputs']

# Cleanup

In [ ]:
# import shutil
# import os
# shutil.rmtree('temp/')
# os.remove('xopt.log')
##os.chmod('dask-worker-space/',0o777)
# shutil.rmtree('dask-worker-space/')
###os.chmod('test.yaml',0o777)
##os.remove('test.yaml')
###os.chmod('NORRAN',0o777)
###os.remove('NORRAN')
#!rm test.yaml

In [ ]:
!rm -r dask-worker-space

In [ ]:
!rm test.yaml

In [ ]:
!rm -r temp

In [ ]:
!rm xopt.log